In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('simdi.csv')
df.head(5)

,kategori,temiz_haber
0,arts life,even death artist even death artist from paris...
1,world,reopens travellers reopens travellers gondolas...
2,arts life,alamy photoalamy photorestoration works ongoin...
3,economy,consumer prices consumer prices mays annual a ...
4,economy,btc oil btc oil the oil delivered kilometerlo...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40126 entries, 0 to 40125
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   kategori     40126 non-null  object
 1   temiz_haber  40126 non-null  object
dtypes: object(2)
memory usage: 627.1+ KB


In [3]:
df.describe().round(1)

,kategori,temiz_haber
count,40126,40126
unique,10,27220
top,turkey,aa photoaa photo
freq,12751,2269


In [4]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
kategori,0,0.0
temiz_haber,0,0.0


In [5]:
df= df.dropna()
df.shape

(40126, 2)

In [6]:
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ozge.altunkaya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ozge.altunkaya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ozge.altunkaya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ozge.altunkaya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [8]:
df['stopwords'] = df['temiz_haber'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['temiz_haber','stopwords']].head()


,temiz_haber,stopwords
0,even death artist even death artist from paris...,7
1,reopens travellers reopens travellers gondolas...,13
2,alamy photoalamy photorestoration works ongoin...,24
3,consumer prices consumer prices mays annual a ...,2
4,btc oil btc oil the oil delivered kilometerlo...,2


In [9]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['temiz_haber'].apply(lambda x: count_punct(x))

In [10]:
df[['temiz_haber','punctuation']].head()

,temiz_haber,punctuation
0,even death artist even death artist from paris...,0
1,reopens travellers reopens travellers gondolas...,0
2,alamy photoalamy photorestoration works ongoin...,0
3,consumer prices consumer prices mays annual a ...,0
4,btc oil btc oil the oil delivered kilometerlo...,0


In [11]:
df['hastags'] = df['temiz_haber'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['temiz_haber','hastags']].head()

,temiz_haber,hastags
0,even death artist even death artist from paris...,0
1,reopens travellers reopens travellers gondolas...,0
2,alamy photoalamy photorestoration works ongoin...,0
3,consumer prices consumer prices mays annual a ...,0
4,btc oil btc oil the oil delivered kilometerlo...,0


In [12]:
df.hastags.loc[df.hastags != 0].count()

0

In [13]:
df['numerics'] = df['temiz_haber'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['temiz_haber','numerics']].head()

,temiz_haber,numerics
0,even death artist even death artist from paris...,0
1,reopens travellers reopens travellers gondolas...,0
2,alamy photoalamy photorestoration works ongoin...,0
3,consumer prices consumer prices mays annual a ...,0
4,btc oil btc oil the oil delivered kilometerlo...,0


In [14]:
df['upper'] = df['temiz_haber'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['temiz_haber','upper']].head()

,temiz_haber,upper
0,even death artist even death artist from paris...,0
1,reopens travellers reopens travellers gondolas...,0
2,alamy photoalamy photorestoration works ongoin...,0
3,consumer prices consumer prices mays annual a ...,0
4,btc oil btc oil the oil delivered kilometerlo...,0


In [15]:
df['baslik'] = df['temiz_haber'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['baslik'].head()

0    even death artist even death artist from paris...
1    reopens travellers reopens travellers gondolas...
2    alamy photoalamy photorestoration works ongoin...
3    consumer prices consumer prices mays annual a ...
4    btc oil btc oil the oil delivered kilometerlon...
Name: baslik, dtype: object

In [16]:
df['baslik'] = df['baslik'].str.replace('[^\w\s]','')
df['baslik'].head()

0    even death artist even death artist from paris...
1    reopens travellers reopens travellers gondolas...
2    alamy photoalamy photorestoration works ongoin...
3    consumer prices consumer prices mays annual a ...
4    btc oil btc oil the oil delivered kilometerlon...
Name: baslik, dtype: object

In [17]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['baslik'] = df['baslik'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['baslik'].sample(10)

741      senior delegation ankara soon travel moscow su...
28214    four people suspected attacking four people su...
15513                           reuters photoreuters photo
19771    employees transport tyres factory zhejiang pro...
8677     greek authorities released custody april one e...
37335    guns n roses singer axl rose left watches acdc...
7343     environment urbanization minister sept said tu...
8361     turkish airlines would purchasing half polant ...
32867    hundreds scavengers make living collecting gar...
14505    istanbul seriously affected says citys mayor t...
Name: baslik, dtype: object

In [18]:
from bs4 import BeautifulSoup

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

strip_html_tags('sdasdasdasd http://t.co/7AzE4IoGMe Risk Assessmen ')

# I will come back to this later

'sdasdasdasd http://t.co/7AzE4IoGMe Risk Assessmen '

In [19]:
from textblob import TextBlob
df['baslik'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    even death artist even death artist paris olde...
1    ropes travellers ropes travellers gondolas rea...
2    alamo photoalamy photorestoration works going ...
3    consumer prices consumer prices may annual gro...
4    etc oil etc oil oil delivered kilometerlong et...
Name: baslik, dtype: object

In [20]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [21]:
df['baslik'] = df.baslik.apply(round1)
df.baslik

0        even death artist even death artist pariss old...
1        reopens travellers reopens travellers gondolas...
2        alamy photoalamy photorestoration works ongoin...
3        consumer prices consumer prices mays annual gr...
4        btc oil btc oil oil delivered kilometerlong bt...
                               ...                        
40121                                   dha photodha photo
40122    burak yılmaz scored six goals champions league...
40123                           reuters photoreuters photo
40124    residents gather site truck bomb attack crowde...
40125    handout picture taken july released nigerian a...
Name: baslik, Length: 40126, dtype: object

In [22]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [23]:
df['baslik'] = df.baslik.apply(round2)
df.baslik

0        even death artist even death artist pariss old...
1        reopens travellers reopens travellers gondolas...
2        alamy photoalamy photorestoration works ongoin...
3        consumer prices consumer prices mays annual gr...
4        btc oil btc oil oil delivered kilometerlong bt...
                               ...                        
40121                                   dha photodha photo
40122    burak yılmaz scored six goals champions league...
40123                           reuters photoreuters photo
40124    residents gather site truck bomb attack crowde...
40125    handout picture taken july released nigerian a...
Name: baslik, Length: 40126, dtype: object

In [24]:
freq = pd.Series(' '.join(df['baslik']).split()).value_counts()[:20]
freq

photo        33456
said         21206
turkish      13204
afp          10512
turkey        9567
aa            8783
minister      8448
president     8320
also          8244
reuters       8091
us            7898
ap            6890
new           6253
people        6236
first         5389
one           5323
percent       5207
two           5199
dha           4905
istanbul      4493
dtype: int64

In [25]:
df['word_count'] = df['baslik'].apply(lambda x: len(str(x).split(" ")))
df[['baslik','word_count']].head()

,baslik,word_count
0,even death artist even death artist pariss old...,146
1,reopens travellers reopens travellers gondolas...,277
2,alamy photoalamy photorestoration works ongoin...,236
3,consumer prices consumer prices mays annual gr...,55
4,btc oil btc oil oil delivered kilometerlong bt...,81


In [26]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
kategori,0,0.0
temiz_haber,0,0.0
stopwords,0,0.0
punctuation,0,0.0
hastags,0,0.0
numerics,0,0.0
upper,0,0.0
baslik,0,0.0
word_count,0,0.0


In [27]:
df['char_count'] = df['baslik'].str.len() ## this also includes spaces
df[['baslik','char_count']].head()

,baslik,char_count
0,even death artist even death artist pariss old...,1065
1,reopens travellers reopens travellers gondolas...,2059
2,alamy photoalamy photorestoration works ongoin...,1834
3,consumer prices consumer prices mays annual gr...,427
4,btc oil btc oil oil delivered kilometerlong bt...,578


In [28]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [29]:
df['avg_word'] = df['baslik'].apply(lambda x: avg_word(x)).round(1)
df[['baslik','avg_word']].head()

,baslik,avg_word
0,even death artist even death artist pariss old...,6.3
1,reopens travellers reopens travellers gondolas...,6.4
2,alamy photoalamy photorestoration works ongoin...,6.8
3,consumer prices consumer prices mays annual gr...,6.8
4,btc oil btc oil oil delivered kilometerlong bt...,6.1


In [30]:
df.sample(2)

,kategori,temiz_haber,stopwords,punctuation,hastags,numerics,upper,baslik,word_count,char_count,avg_word
27141,turkey,aa photoaa photo,0,0,0,0,0,aa photoaa photo,3,16,4.7
16684,world,pope benedict xvi backc speaks bishop holds ta...,2,0,0,0,0,pope benedict xvi backc speaks bishop holds ta...,41,290,6.1


In [31]:
df['baslik'] = df.baslik.apply(round1)
df.baslik

0        even death artist even death artist pariss old...
1        reopens travellers reopens travellers gondolas...
2        alamy photoalamy photorestoration works ongoin...
3        consumer prices consumer prices mays annual gr...
4        btc oil btc oil oil delivered kilometerlong bt...
                               ...                        
40121                                   dha photodha photo
40122    burak yılmaz scored six goals champions league...
40123                           reuters photoreuters photo
40124    residents gather site truck bomb attack crowde...
40125    handout picture taken july released nigerian a...
Name: baslik, Length: 40126, dtype: object

In [33]:
df['baslik'] = df.temiz_haber.apply(round2)
df.temiz_haber

0        even death artist even death artist from paris...
1        reopens travellers reopens travellers gondolas...
2        alamy photoalamy photorestoration works ongoin...
3        consumer prices consumer prices mays annual a ...
4        btc oil btc oil the oil delivered  kilometerlo...
                               ...                        
40121                                   dha photodha photo
40122    burak yılmaz scored six goals champions league...
40123                           reuters photoreuters photo
40124    residents gather site truck bomb attack crowde...
40125    an handout picture taken july   released niger...
Name: temiz_haber, Length: 40126, dtype: object

In [32]:
df.to_csv('tertemiz.csv', index=False)

In [16]:
import pandas as pd
import numpy as np
df = pd.read_csv('simdi.csv')
df.head(5)


,kategori,temiz_haber
0,arts life,even death artist even death artist from paris...
1,world,reopens travellers reopens travellers gondolas...
2,arts life,alamy photoalamy photorestoration works ongoin...
3,economy,consumer prices consumer prices mays annual a ...
4,economy,btc oil btc oil the oil delivered kilometerlo...


In [18]:
from textblob import TextBlob
df['polarity'] = df[['temiz_haber']].sample(12205)

def detect_subjectivity(temiz_haber):
    return TextBlob(temiz_haber).sentiment.subjectivity
df['subjectivity'] = df.temiz_haber.apply(detect_subjectivity)

def detect_polarity(temiz_haber):
    return TextBlob(temiz_haber).sentiment.polarity
df['polarity'] = df.temiz_haber.apply(detect_polarity)     


def f(df):
    if df['polarity'] > 0:
        val = 1
    elif df['polarity'] == 0:
        val = 0
    else:
        val = -1
    return val
df['sentiment'] = df.apply(f, axis=1)


In [19]:
df.polarity

0       -0.011626
1       -0.034202
2        0.076139
3        0.107273
4       -0.033333
           ...   
40121    0.000000
40122    0.500000
40123    0.000000
40124    0.000000
40125   -0.100000
Name: polarity, Length: 40126, dtype: float64

In [20]:
df.head(5)



,kategori,temiz_haber,polarity,subjectivity,sentiment
0,arts life,even death artist even death artist from paris...,-0.011626,0.229196,-1
1,world,reopens travellers reopens travellers gondolas...,-0.034202,0.351643,-1
2,arts life,alamy photoalamy photorestoration works ongoin...,0.076139,0.390114,1
3,economy,consumer prices consumer prices mays annual a ...,0.107273,0.234242,1
4,economy,btc oil btc oil the oil delivered kilometerlo...,-0.033333,0.365385,-1


In [21]:
df.to_csv('ensonhali.csv', index=False)